In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn

# Load Dataset 


In [10]:
# Let's load the dataset
def load_data():

    data = pd.read_csv('onnx-example-main1/data/synth_data_for_training.csv')

    # Let's specify the features and the target
    y = data['checked']
    X = data.drop(['checked'], axis=1)
    X = X.astype(np.float32)

    # Let's split the dataset into train and test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

    return X_train, X_test, y_train, y_test

# Loading and Running on Test Model

In [11]:
# for running tests, use this
def load_onnx_model(model_path):
    sess = rt.InferenceSession(str(model_path))
    return sess

In [12]:
def predict(sess, X) -> np.ndarray:
    input = sess.get_inputs()[0].name
    # if you run into issues do X.values.astype("float32")
    outputs = sess.run(None, {input: X})
    predictions = outputs[0]

    if predictions.ndim == 2 and predictions.shape[1] == 2:
        return predictions[:,1]
    return predictions.astype("float32")


In [13]:
def predict_labels(sess, X) -> np.ndarray:
    labels = predict(sess, X)
    return (labels >= 0.5).astype(int)

# Partitioning Tests

In [14]:
# age, gender, and language proxies

# gender

def test_gender_accuracy_similar():
    _ , X_test, _ , y_test = load_data()
    sess = load_onnx_model()

    GENDER_COL = "persoon_geslacht_vrouw"

    assert GENDER_COL in X_test.columns, f"{GENDER_COL} not in dataset"

    filter_empty = X_test[GENDER_COL].notna()
    X_test = X_test[filter_empty]
    y_test = y_test[filter_empty]

    women = X_test[GENDER_COL] == 1
    other = X_test[GENDER_COL] == 0

    # remove if not needed
    assert women.sum() > 40, "Not enough women!"
    assert other.sum() > 40, "Not enough others!"

    y_pred = predict_labels(sess, X_test)

    women_accuracy = accuracy_score(y_test[women], y_pred[women])
    other_accuracy = accuracy_score(y_test[other], y_pred[other])

    difference = abs(women_accuracy - other_accuracy)
    print(f"[Partition gender] women_accuracy={women_accuracy:.3f}, other_accuracy={other_accuracy:.3f}, diff={difference:.3f}")

    tolerance = 0.18
    assert difference<=tolerance, (f"{difference:.3f} > {tolerance:.3f}. Difference is too large.")




In [ ]:
# age, gender, and language proxies

# gender

def test_gender_rate():
    _ , X_test, _ , _ = load_data()
    sess = load_onnx_model()

    GENDER_COL = "persoon_geslacht_vrouw"

    assert GENDER_COL in X_test.columns, f"{GENDER_COL} not in dataset"

    filter_empty = X_test[GENDER_COL].notna()
    X_test = X_test[filter_empty]

    women = X_test[GENDER_COL] == 1
    other = X_test[GENDER_COL] == 0

    # remove if not needed
    assert women.sum() > 40, "Not enough women!"
    assert other.sum() > 40, "Not enough others!"

    y_pred = predict_labels(sess, X_test)

    women_rate = y_pred[women].mean()
    other_rate = y_pred[other].mean()

    difference = abs(women_rate - other_rate)
    print(f"[Partition gender] women_rate={women_rate:.3f}, other_rate={other_rate:.3f}, diff={difference:.3f}")

    tolerance = 0.22
    assert difference<=tolerance, (f"{difference:.3f} > {tolerance:.3f}. Difference is too large.")




In [15]:
# ============================================================
# Metamorphic Data Augmentation: Gender Flipping
# ============================================================

# Gender transformation map (same as above)
gender_flip_map = {
    1: 2,
    2: 1,
    3: 5,
    5: 3
}

def flip_gender_all(val):
    """Flip gender if possible, otherwise return original."""
    return gender_flip_map.get(val, val)


# --------------------------
# Create flipped training set
# --------------------------
X_train_flipped = X_train.copy()
X_train_flipped["Attribute9"] = X_train_flipped["Attribute9"].apply(flip_gender_all)

# y_train does not change
y_train_flipped = y_train.copy()

# --------------------------
# Augment the training data
# --------------------------
X_train_aug = pd.concat([X_train, X_train_flipped], ignore_index=True)
y_train_aug = pd.concat([y_train, y_train_flipped], ignore_index=True)

print("Original training size:", len(X_train))
print("Augmented training size:", len(X_train_aug))

NameError: name 'X_train' is not defined

In [ ]:
clf_aug = DecisionTreeClassifier()
clf_aug.fit(X_train_aug, y_train_aug)

In [ ]:
# Create a copy of test set
X_test_flipped = X_test.copy()

# Apply to Attribute9 (column 8)
X_test_flipped['Attribute9'] = X_test_flipped['Attribute9'].apply(flip_gender_all)

# Predict with flipped gender
y_pred_original = clf_aug.predict(X_test)
y_pred_flipped = clf_aug.predict(X_test_flipped)

# Compare
print("Accuracy after gender flip:", metrics.accuracy_score(y_test, y_pred_flipped))
print("Changed predictions:", np.sum(y_pred_original != y_pred_flipped))
